# SARIMA Forecasting with Electricity Demand Data (EIA API)
This Colab notebook uses U.S. electricity demand data from the EIA API (PJM region) to train and forecast using a SARIMA model.

In [ ]:
!pip install pandas matplotlib statsmodels requests


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Paste your EIA API key here
api_key = "YOUR_EIA_API_KEY"


In [ ]:
url = "https://api.eia.gov/v2/electricity/rto/region-data/data/"
params = {
    "api_key": api_key,
    "frequency": "hourly",
    "data[0]": "value",
    "facets[respondent][]": "PJM",
    "facets[type][]": "D",
    "start": "2025-01-01T00",
    "end": datetime.now().strftime("%Y-%m-%dT%H")
}

response = requests.get(url, params=params)
data = response.json()
df = pd.DataFrame(data["response"]["data"])
df["period"] = pd.to_datetime(df["period"])
df = df.sort_values("period").set_index("period")
df = df.rename(columns={"value": "demand_MWh"})
df["demand_MWh"] = pd.to_numeric(df["demand_MWh"], errors="coerce")
df = df.dropna(subset=["demand_MWh"])
df.head()


In [ ]:
daily_df = df.resample("D").mean()
daily_df = daily_df.fillna(method="ffill")
daily_df.tail()


In [ ]:
plt.figure(figsize=(14,5))
plt.plot(daily_df, label='Daily Avg Demand (MWh)')
plt.title('PJM Electricity Demand – 2025')
plt.xlabel('Date')
plt.ylabel('Megawatt-Hours')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
y = daily_df['demand_MWh']
model = SARIMAX(y, order=(1,1,1), seasonal_order=(1,1,1,7))
results = model.fit()
print(results.summary())


In [ ]:
forecast = results.forecast(14)
plt.figure(figsize=(14,5))
plt.plot(y[-60:], label="Actual")
plt.plot(forecast, label="Forecast", linestyle='--')
plt.title("Forecast of Electricity Demand (Next 14 Days)")
plt.xlabel("Date")
plt.ylabel("Megawatt-Hours")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
daily_df.to_csv("pjm_daily_demand_2025.csv")
forecast.to_csv("pjm_forecast_next14days.csv")
